In [1]:
import pandas as pd
import numpy as np
from pprint import pprint

In [2]:
data = [['Orange', 'big','Orange','sweet','Yes'], 
        ['Apple', 'big','Red-Brown','sweet','Yes'],   
        ['Orange', 'big','Orange','sweet','Yes'], 
        ['Apple', 'big','Red-Brown','sweet','Yes'],  
        ['Orange', 'big','Orange','sweet','Yes'], 
        ['Apple', 'big','Red-Brown','sweet','Yes'],   
        ['Orange', 'big','Orange','sweet','Yes'],
        ['Apple', 'big','Red-Brown','sweet','Yes'],
        ['Grape', 'small','Red-Brown','sour','Yes'],
        ['Grape', 'small','Red-Brown','sour','Yes'],
        ['Grape', 'small','Red-Brown','sweet','Yes'],
        ['Grape', 'small','Orange','sweet','No'],
        ['Grape', 'small','Orange','sweet','No'],
        ['lemon', 'medium','Red-Brown','sour','No'],
        ['lemon', 'medium','Red-Brown','sour','No'],
        ['lemon', 'medium','Red-Brown','sour','No'],
        ['lemon', 'medium','Red-Brown','sour','No']] 
  
df = pd.DataFrame(data, columns = ['fruit','size', 'color','taste','eat']) 

In [3]:
df

,fruit,size,color,taste,eat
0,Orange,big,Orange,sweet,Yes
1,Apple,big,Red-Brown,sweet,Yes
2,Orange,big,Orange,sweet,Yes
3,Apple,big,Red-Brown,sweet,Yes
4,Orange,big,Orange,sweet,Yes
5,Apple,big,Red-Brown,sweet,Yes
6,Orange,big,Orange,sweet,Yes
7,Apple,big,Red-Brown,sweet,Yes
8,Grape,small,Red-Brown,sour,Yes
9,Grape,small,Red-Brown,sour,Yes


In [4]:
df.describe()

,fruit,size,color,taste,eat
count,17,17,17,17,17
unique,4,3,2,2,2
top,Grape,big,Red-Brown,sweet,Yes
freq,5,8,11,11,11


In [5]:
# dropping the fruit name column as i dont want to descriminate any fruit , if you want you can consider this column  as well

dataset=df.drop('fruit',axis=1)

In [6]:
# entropy function  -plogp -qlogq  

def entropy(target_col):
    elements,counts = np.unique(target_col,return_counts = True)
    entropy = np.sum([(-counts[i]/np.sum(counts))*np.log2(counts[i]/np.sum(counts)) for i in range(len(elements))])
    return entropy

In [7]:
# lets see the entropy of the class(depenednt variable) eat

total_entropy_for_decision_eat =entropy(dataset['eat'])
total_entropy_for_decision_eat

0.9366673818775626

In [8]:
#lets calculate the entropy and information gain of other features

sweetCount = dataset[dataset['taste']=="sweet"]["eat"].count()
SweetCountWeightedAverage = sweetCount/dataset["eat"].count()


sourCount = dataset[dataset['taste']=="sour"]["eat"].count()
SourCountWeightedAverage = sourCount/dataset["eat"].count()


total_entropy_for_taste =SweetCountWeightedAverage * entropy(dataset[dataset['taste']=="sweet"]["eat"])+SourCountWeightedAverage * entropy(dataset[dataset['taste']=="sour"]["eat"])
print("entropy for taste :{0}".format(total_entropy_for_taste))

informationGainForTaste= total_entropy_for_decision_eat - total_entropy_for_taste
print("information gain for taste :{0}".format(informationGainForTaste))

entropy for taste :0.7667175174327292
information gain for taste :0.1699498644448334


In [ ]:
#lets genaralizing the above
#lets calculate the information gain

In [9]:
def getInformationGain(currentColumn, targetColumn):
    elements,counts = np.unique(dataset[currentColumn],return_counts = True)
    totalEntropyForCurrentColumn= np.sum([(counts[i]/np.sum(counts))*entropy(dataset[dataset[currentColumn]==elements[i]][targetColumn]) for i in range(len(elements))])
    return entropy(dataset[targetColumn]) - totalEntropyForCurrentColumn    

In [10]:
print("information gain for taste :{0}".format(getInformationGain("taste","eat")))
print("information gain for colorIndex :{0}".format(getInformationGain("color","eat")))
print("information gain for size :{0}".format(getInformationGain("size","eat")))

information gain for taste :0.1699498644448334
information gain for colorIndex :0.0006651257049756776
information gain for size :0.6510936776261895


In [190]:
# in the above case we got highest information gain on the column size, so it will become the root node
# root node can be split into number of child nodes matching to distinct fields inside the root node columns
# to make a tree from root node we need a queue

In [11]:
#generalize the above information gain code

def getHighestInformationGainColumn(targetColumn,features):
    ig=0
    column=None
    for i in features:
        newIg= getInformationGain(i,targetColumn)
        if(newIg>ig):
            ig=newIg
            column=i
    return column

In [13]:
getHighestInformationGainColumn('eat',['taste','color','size'])

'size'

In [17]:
# create Tree recursively

def createTree(data,originaldata,features,targetColumn,parent_node=None):
    if len(np.unique(data[targetColumn])) <= 1:
        return np.unique(data[targetColumn])[0]
    elif len(data)==0:
        return np.unique(originaldata[targetColumn])[np.argmax(np.unique(originaldata[targetColumn],return_counts=True)[1])]
    elif len(features) ==0:
        return parent_node
    else: 
        parent_node = np.unique(data[targetColumn])[np.argmax(np.unique(data[targetColumn],return_counts=True)[1])]
        bestFeature= getHighestInformationGainColumn(targetColumn,features)   
        tree = {bestFeature:{}}
        features = [i for i in features if i != bestFeature]
        for value in np.unique(data[bestFeature]):   
                newData = data.where(data[bestFeature] == value).dropna()         
                subtree = createTree(newData,originaldata,features,targetColumn,parent_node)         
                tree[bestFeature][value] = subtree            
        return(tree)  

In [18]:
tree=createTree(dataset,dataset,dataset.columns[:-1],"eat")

In [19]:
tree

{'size': {'big': 'Yes',
  'medium': 'No',
  'small': {'taste': {'sour': 'Yes',
    'sweet': {'color': {'Orange': 'No', 'Red-Brown': 'Yes'}}}}}}